<a href="https://colab.research.google.com/github/D-2000-99/Tensorflow-Object-Detection/blob/main/SSD_video_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Single Shot Detector COCO(Video)

In this project we attempt to implement object detection in a video format

#Imports

## Cloning TF Model Repo

In [ ]:
#Cloning TF repo with all models
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 68232, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 68232 (delta 3), reused 9 (delta 2), pack-reused 68221
Receiving objects: 100% (68232/68232), 576.50 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (47928/47928), done.


##Protobuf

In [ ]:
#Preinstalled in Colab
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

## Installing Object Detection API

In [ ]:
#Running models/research/object_detection/packages/tf2/setup.py
!cd models/research && \
  cp object_detection/packages/tf2/setup.py . && \
  python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 5.7 MB/s 
     |████████████████████████████████| 352 kB 49.3 MB/s 
     |████████████████████████████████| 1.8 MB 38.0 MB/s 
     |████████████████████████████████| 23.1 MB 43.0 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 90 kB 8.8 MB/s 
     |████████████████████████████████| 4.9 MB 48.9 MB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 213 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 41.

##Importing base libraries

In [ ]:
#File/Image handling
import os
from PIL import Image
import imageio #For video
import pathlib #For Path
import time #measure processing time

#Essential DL packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#Object-Detection packages 
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils #For annotation of Video

In [ ]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##Downloading Video

In [ ]:
!wget -nc https://static.videezy.com/system/resources/previews/000/020/763/original/P1033666.mp4 #-nc : -no--clobber; enables overwrite 

--2021-12-28 08:06:10--  https://static.videezy.com/system/resources/previews/000/020/763/original/P1033666.mp4
Resolving static.videezy.com (static.videezy.com)... 104.19.138.77, 104.19.142.77, 104.19.140.77, ...
Connecting to static.videezy.com (static.videezy.com)|104.19.138.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409909 (1.3M) [video/mp4]
Saving to: ‘P1033666.mp4’

P1033666.mp4        100%[===================>]   1.34M  --.-KB/s    in 0.02s   

2021-12-28 08:06:10 (82.0 MB/s) - ‘P1033666.mp4’ saved [1409909/1409909]



In [ ]:
INPUT_VIDEO = ['P1033666']

In [ ]:
#url = 'http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_kpts.tar.gz' #MobileNet has
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin=url,
    untar=True
)

386539520/386527459 [==============================] - 4s 0us/step


In [ ]:
PATH_TO_MODEL_DIR

'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

In [ ]:
!cd /root/.keras/datasets && ls

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz


In [ ]:
!cd  /root/.keras/datasets/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 && ls

/bin/bash: line 0: cd: /root/.keras/datasets/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8: No such file or directory


In [ ]:
#MobileNet extracted under different name
#PATH_TO_MODEL_DIR = '/root/.keras/datasets/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'

##Getting Label Files

In [ ]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url,
    extract=False
)

16384/5056 [=================================================================================================] - 0s 0us/step


In [ ]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [ ]:
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


##Loading model

In [ ]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
#Sanity check for saved_model folder
!cd {PATH_TO_SAVED_MODEL} && ls

assets	saved_model.pb	variables


In [ ]:
print('Starting model loading...', end='')
start_time = time.time()

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL) #tf.saved_model.load returns a function, detect_fn expects input tensor

end_time = time.time()

elapsed = end_time - start_time
print(f'and done in {elapsed} s')

Starting model loading...and done in 41.608848094940186 s


##Loading labels

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(
    #This function is imported from object_detection.utils
    PATH_TO_LABELS,
    use_display_name= True
)

In [ ]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

##Object detection code

In [ ]:
def detect_objects_in_image(image_np):
  #Input section
  input_tensor = tf.convert_to_tensor(image_np)#Converting to required input tensor
  input_tensor = input_tensor[tf.newaxis, ...]#Model requires batch of images (batch tensor), so adding new axis to mimic
  detections = detect_fn(input_tensor)

  #Output section
  #Convert output batch tensors to np array (take index[0] to remove batch dimension)
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy() #.numpy() - Converts tf/pytorch tensor to np array
                  for key, value in detections.items()}
  #Casting detection_classes as int64                 
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


  #Applying Bounding boxes and labels to video
  image_np_with_detections = image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,           #Image to annotate, inplace
      detections['detection_boxes'],      #Boxes from detections
      detections['detection_classes'],    #int64 classes np array from detections
      detections['detection_scores'],     #Detection scores from detections
      category_index,                     #Label mappings in specific format from object_detection.utils
      use_normalized_coordinates=True,    #Image resolution normalized 0-1
      max_boxes_to_draw=200,              #Number of boxes on screen
      min_score_thresh=.30,            #Min value for drawing BB
      agnostic_mode=False                 #Possibly class-agnostic mode? Only BB no labels?
  )
  return image_np_with_detections

def detect_objects_in_video(input_video):
  print(f'Detecting objects in video {input_video}.mp4...',end='')

  #imageio set-up
  video_reader = imageio.get_reader(f'{input_video}.mp4')# Assuming only mp4 videos!
  video_writer = imageio.get_writer(f'{input_video}_annotated.mp4', fps=10)

  #looping through and running detection model on each frame
  t0 = time.time()
  n_frames = 0
  for frame in video_reader:
    n_frames += 1
    new_frame = detect_objects_in_image(frame)
    video_writer.append_data(new_frame)# Writing frame to video
  
  fps = n_frames / (time.time() - t0)
  print(f'Frames processed: {n_frames}, Speed: {fps}fps')

  #Clean up
  video_writer.close()

In [ ]:
INPUT_VIDEO[0]

'P1033666'

In [ ]:
#Video Saved to files
time0 = time.time()
detect_objects_in_video(INPUT_VIDEO[0])
time1 = time.time()
elapsed = time1-time0
print(elapsed)

Detecting objects in video P1033666.mp4...

Frames processed: 405, Speed: 2.0537751407194405fps
198.36719632148743
